In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
import joblib

In [6]:
cd store-sales-time-series-forecasting

c:\Users\naosa\Desktop\kaggle\test\store-sales-time-series-forecasting


c:\Users\naosa\Desktop\kaggle\test\myenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
#学習データ
train = pd.read_csv("train.csv")
x = train.drop('sales', axis=1)
x.head()

,id,date,store_nbr,family,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0
1,1,2013-01-01,1,BABY CARE,0
2,2,2013-01-01,1,BEAUTY,0
3,3,2013-01-01,1,BEVERAGES,0
4,4,2013-01-01,1,BOOKS,0


In [8]:
stores = pd.read_csv("stores.csv")
stores.head()
#stores_nbrをキーとしてmerge可能

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
# store_nbrをキーに左結合
merged_x = pd.merge(x, stores, on='store_nbr', how='left')
merged_x = merged_x.drop(['date','id'], axis=1)
merged_x.head()


,store_nbr,family,onpromotion,city,state,type,cluster
0,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,1,BABY CARE,0,Quito,Pichincha,D,13
2,1,BEAUTY,0,Quito,Pichincha,D,13
3,1,BEVERAGES,0,Quito,Pichincha,D,13
4,1,BOOKS,0,Quito,Pichincha,D,13


In [10]:
cat_columns = ['family', 'city', 'state', 'type']

# LabelEncoderを使用してエンコーディング
label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    merged_x[col] = le.fit_transform(merged_x[col].astype(str))
    label_encoders[col] = le

merged_x

,store_nbr,family,onpromotion,city,state,type,cluster
0,1,0,0,18,12,3,13
1,1,1,0,18,12,3,13
2,1,2,0,18,12,3,13
3,1,3,0,18,12,3,13
4,1,4,0,18,12,3,13
...,...,...,...,...,...,...,...
3000883,9,28,0,18,12,1,6
3000884,9,29,1,18,12,1,6
3000885,9,30,148,18,12,1,6
3000886,9,31,8,18,12,1,6


In [11]:
#正解データ
y = train["sales"]
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: sales, dtype: float64

In [12]:
# 負の売上値の確認
negative_sales = (y < 0).sum()
print(f"負の売上値の数: {negative_sales}")

# データの前処理: 負の値を0に置き換える
y = y.clip(lower=0)


負の売上値の数: 0


In [13]:
print(f"特徴量の形状: {merged_x.shape}")
print(f"ターゲットの形状: {y.shape}")


特徴量の形状: (3000888, 7)
ターゲットの形状: (3000888,)


In [14]:
# トレーニングデータとテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(merged_x, y, test_size=0.2, random_state=42)

In [15]:
# LightGBMのパラメータ設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# LightGBMデータセットの作成
train_data = lgb.Dataset(x_train, label=y_train)

# モデルのトレーニング
model = lgb.train(params, train_data, num_boost_round=100)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 352
[LightGBM] [Info] Number of data points in the train set: 2400710, number of used features: 7
[LightGBM] [Info] Start training from score 357.301279


In [16]:
# テストデータでの予測
y_pred = model.predict(x_test)


In [17]:
y_test

2483653    2576.000
2761877       0.000
1217988     357.354
1231938       5.000
528175        0.000
             ...   
2640551       3.000
1201944     258.000
12974       292.884
1136162     738.464
2582492    1412.923
Name: sales, Length: 600178, dtype: float64

In [18]:
# RMSLEの計算（負の値がないことを確認済み）
def rmsle(y_test, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_test), 2)))

rmsle_score = rmsle(y_test, y_pred)
print(f"Root Mean Squared Logarithmic Error: {rmsle_score:.4f}")


Root Mean Squared Logarithmic Error: 2.3376


C:\Users\naosa\AppData\Local\Temp\ipykernel_32064\1350841475.py:3: RuntimeWarning: invalid value encountered in log1p
  return np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_test), 2)))


#モデルの保存

In [19]:

# または、joblib を使用して保存することもできます
joblib.dump(model, 'lightgbm_model.joblib')
# LabelEncoderも保存する
joblib.dump(label_encoders, 'label_encoders.joblib')


['label_encoders.joblib']

In [21]:

# 特徴量名の保存
import json
with open('feature_names.json', 'w') as f:
    json.dump(list(merged_x.columns), f)